# Experiment and path specification

In [ ]:
# Specify the experiment for analysis
experiment='cxix40218'
runNumber = 58

# Set the installation and output path
import os
os.environ['INSTALLPATH']= '/cds/home/i/igabalsk/TRXS-Run18'
os.environ['OUTPUTPATH']= '/cds/data/psdm/%s/%s/scratch' % (experiment[0:3],experiment)

# Import Libraries

In [ ]:
# Magic iPython command to enable plotting
%matplotlib inline

# Load in the pythonBatchMagic library
import sys
print os.environ['INSTALLPATH']+'/Libraries/pythonBatchMagic'
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/pythonBatchMagic')
from pythonBatchMagic import *
# Determine current user
currentUser, error = unixCMD("echo $USER")
print(currentUser)
os.environ['RESULTSPATH']= ('/cds/data/psdm/%s/%s/results/%s' % (experiment[0:3],experiment,currentUser)).strip()
if not os.path.exists(os.environ['RESULTSPATH']):
    os.mkdir(os.environ['RESULTSPATH'])

# Leveraging the batch queue to quickly grab detector data

## Loading libraries

In [ ]:
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/LCLS')
from LCLSdefault import *

sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/mattsLibraries')
from picklez import *

# Load in the get data library
from dataAnalysis import *

# Load in the batch library for lcls
from lclsBatch import *

## Load point data

In [ ]:
pointData = load_obj( os.environ['RESULTSPATH']+'/pointData-run-%d' % runNumber )

## Plot histogram of result

In [ ]:
detArrays = pointData

fig2, ax2 = plt.subplots(1, 1)
ax2.hist(detArrays['xrayEnergy'], bins=100,normed=False, alpha=0.5);

# Read in CSPAD via batch job

## Specify good idxs

In [ ]:
goodIdx = runFilter( pointData , filterOn=['xrayEnergy', 'ttfltpos'], maddevs=3 ).astype(bool)
print 'Number of good shots: ', len(goodIdx[goodIdx==True])

## Now generate time delays

In [ ]:
TTPoly = np.array([2.235e-7, -2.21e-3, 8.547e-1]) 

TTFltPos = pointData['ttfltpos']
TTCorrFltPos = TTPoly[0]*TTFltPos**2+TTPoly[1]*TTFltPos+TTPoly[2]
TTTime = TTCorrFltPos - 1e9/3e8*2*(pointData['stageencoder']-56.35)

# NOTE: This is a placeholder, and should be removed on runs with timetool data
# TTTime = np.zeros(pointData['xrayEnergy'].shape)

fig2, ax2 = plt.subplots(1, 1)
ax2.hist(TTTime[goodIdx], bins=200,normed=False, alpha=0.5)
ax2.set_xlabel('Delay (ps)')
ax2.set_ylabel('Counts')
plt.show()

# Read in all images
## If you just want to look at all the images at once

In [ ]:
# tagList = []
# size_slice = len(goodIdx[goodIdx==True])
# for idx in range(1):
#     filteredSeconds = pointData['seconds'][goodIdx][0+size_slice*idx:size_slice+size_slice*idx]
#     filteredNanoseconds = pointData['nanoseconds'][goodIdx][0+size_slice*idx:size_slice+size_slice*idx]
#     filteredFiducials = pointData['fiducials'][goodIdx][0+size_slice*idx:size_slice+size_slice*idx]
#     tagList.append( {'seconds':filteredSeconds, 'nanoseconds':filteredNanoseconds, 'fiducials':filteredFiducials} )

# # batchThreads = batchCSPADGrabber (tagList, experiment=experiment, runNumber=runNumber, detType='Jungfrau')
# batchThreads = batchCSPADMVGrabber (tagList, experiment=experiment, runNumber=runNumber, detType='Jungfrau')
# timebins=np.arange(10)
# batchThreads.Queue = 'psfehhiprioq'   
# batchThreads.start()

# Read in timebinned CSPAD
## If you want to timebin your images

In [ ]:
# timebins = np.arange( -1.5,1.5,.03 )
# timebins are in picoseconds
# timebins = np.arange( 2.,5.5,.03 ) 
timebins = np.arange( -5.5, 5.5, .1 )
print 'Number of timebins: ', len(timebins)
dt = timebins[1]-timebins[0]
NT = timebins.size

tagList = []
for idx, t in enumerate(timebins):
    tIndex = (TTTime >= t-dt/2.)&(TTTime < t+dt/2.)
    allIndex = tIndex & goodIdx
    filteredSeconds = pointData['seconds'][allIndex]
    filteredNanoseconds = pointData['nanoseconds'][allIndex]
    filteredFiducials = pointData['fiducials'][allIndex]
    tagList.append( {'seconds':filteredSeconds, 'nanoseconds':filteredNanoseconds, 'fiducials':filteredFiducials} )

# batchThreads = batchCSPADGrabber (tagList, experiment=experiment, runNumber=runNumber, detType='Jungfrau')
batchThreads = batchCSPADMVGrabber (tagList, experiment=experiment, runNumber=runNumber, detType='Jungfrau')

batchThreads.Queue = 'psfehhiprioq'
batchThreads.RunType = 'mpirun python2'
batchThreads.start()


## Check job status

In [ ]:
print batchThreads.status
stdoutdata, stderrdata = bjobs()
print stdoutdata
print stderrdata

## Kill threads if they die

In [ ]:
batchThreads.requestStop()
time.sleep(5)
stdoutdata, stderrdata = bkill(killAll=True)
print stdoutdata
print stderrdata

# Save resulting CSPAD

In [ ]:
batchThreads.gather()
CSPAD = batchThreads.CSPAD
variance = batchThreads.variance
counts = batchThreads.counts

In [ ]:

np.save(os.environ['RESULTSPATH']+'/timebins-run-%d' % runNumber ,timebins)
np.save(os.environ['RESULTSPATH']+'/CSPAD-run-%d' % runNumber  , CSPAD)
np.save(os.environ['RESULTSPATH']+'/variance-run-%d' %  runNumber , variance)
np.save(os.environ['RESULTSPATH']+'/counts-run-%d' % runNumber  , counts)


# Plot CSPAD

In [ ]:
# # Optional code if timebinned frames are already saved
timebins = np.load(os.environ['RESULTSPATH']+'/timebins-run-%d.npy' % runNumber )
CSPAD = np.load(os.environ['RESULTSPATH']+'/CSPAD-run-%d.npy' % runNumber)
variance = np.load(os.environ['RESULTSPATH']+'/variance-run-%d.npy' % runNumber)
counts = np.load(os.environ['RESULTSPATH']+'/counts-run-%d.npy' % runNumber)

In [ ]:
for i in range(CSPAD.shape[-1]):
    m = np.mean(CSPAD[:,:,:,i])
    if not np.isnan(m):
        print i, np.mean(CSPAD[:,:,:,i]), counts[i][0], timebins[i]

In [ ]:
from IPython.display import clear_output
from plotStyles import *

def plotCSPAD( cspad , x , y, cspadMask=None, zLims = None, divergent=False, NTILE=8 ):
    figOpts = {'xLims':[-1e5,1e5],'yLims':[-1e5,1e5],'divergent':divergent, 'xIn':3, 'yIn':3*11.5/14.5}
    
    if zLims is not None:
        figOpts['zLims'] = zLims
    
    for iTile in range(NTILE):
    
        if cspadMask is not None:
            cspadTile = cspad[iTile,:,:]
            tileMask = ~cspadMask[iTile,:,:]
            cspadTile[tileMask] = 0
        
        if iTile == 0:
            newFigure = True
        else:
            newFigure = False
            
        clear_output()
        colorPlot( x[iTile,:,:], y[iTile,:,:], cspadTile , newFigure=newFigure, **figOpts);


x,y = CSPADgeometry(detType='Jungfrau', run=runNumber, experiment=experiment)
# cspadMask = createMask(experiment=experiment, run=runNumber, detType='Jungfrau').astype(bool)
cspadMask = np.ones_like(x).astype(bool)
print(cspadMask.shape)

print CSPAD.shape
CSPADbinned = CSPAD

# CSPADbinned[CSPADbinned>10]=0
CSPAD2plot = np.nansum(CSPADbinned[:,:,:,:],axis=-1)#-CSPADbinned[:,:,:,53]
# plotCSPAD( cspadMask, x , y , cspadMask=cspadMask, divergent=True )
# plotCSPAD( np.sum(CSPADbinned[:,:,:,:100], axis=-1)-np.sum(CSPADbinned[:,:,:,100:200], axis=-1), x , y , cspadMask=cspadMask, divergent=False, NTILE=8 )
plotCSPAD( CSPAD2plot, x , y , zLims=[0,2],
          cspadMask=cspadMask, divergent=False, NTILE=8 )
